In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

In [2]:
class WineDataset(Dataset):
    def __init__(self, path):
        # data loading
        xy = np.loadtxt(path, delimiter=',', dtype=np.float32, skiprows=1)
        self.X = torch.from_numpy(xy[:, 0:-1])
        self.y = torch.from_numpy(xy[:, -1]).view([-1,1])
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        # dataset[0]
        return self.X[index], self.y[index]
        
    def __len__(self):
        # len(dataset)
        return self.n_samples

In [3]:
ds_train = WineDataset('lecture_07_train.csv')
ds_test = WineDataset('lecture_07_test.csv')
train_dataloader = DataLoader(dataset=ds_train, batch_size=4, shuffle=True)
test_dataloader = DataLoader(dataset=ds_test, batch_size=1, shuffle=False)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        pred = self.linear_relu_stack(x)
        return pred

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

In [6]:
learning_rate = 0.0001
model = NeuralNetwork()
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 75.259491  [    4/ 3918]
loss: 6.828148  [  404/ 3918]
loss: 1.104428  [  804/ 3918]
loss: 2.881206  [ 1204/ 3918]
loss: 2.337165  [ 1604/ 3918]
loss: 1.189135  [ 2004/ 3918]
loss: 1.564875  [ 2404/ 3918]
loss: 2.931337  [ 2804/ 3918]
loss: 1.696848  [ 3204/ 3918]
loss: 0.911774  [ 3604/ 3918]
Avg loss: 1.697283 

Epoch 2
-------------------------------
loss: 1.013132  [    4/ 3918]
loss: 1.035671  [  404/ 3918]
loss: 0.131800  [  804/ 3918]
loss: 0.352381  [ 1204/ 3918]
loss: 0.467473  [ 1604/ 3918]
loss: 0.539987  [ 2004/ 3918]
loss: 1.776217  [ 2404/ 3918]
loss: 0.901376  [ 2804/ 3918]
loss: 2.305511  [ 3204/ 3918]
loss: 1.438245  [ 3604/ 3918]
Avg loss: 0.859446 

Epoch 3
-------------------------------
loss: 0.484519  [    4/ 3918]
loss: 0.325553  [  404/ 3918]
loss: 0.420671  [  804/ 3918]
loss: 0.126817  [ 1204/ 3918]
loss: 0.234268  [ 1604/ 3918]
loss: 0.964683  [ 2004/ 3918]
loss: 0.831622  [ 2404/ 3918]
loss: 0.182909  [ 2804/ 391